# Pb_Multiphase_Enthalpie

In [ ]:
from trustutils import run

run.introduction("Elie Saikali & Adrien Bruneton")
run.TRUST_parameters()

In [ ]:
import pandas as pd
import os, sys
from string import Template

run.reset()
run.initBuildDirectory()

dis = [("VDF", "", "-"), ("VEF", "Trianguler_H dom_solide Trianguler_H dom_fluide", "--")]
diffusion = [("Diff_explicite", "0"), ("Diff_implicite", "1")]
schema = [
          ("Euler", "Scheme_euler_explicit", "1") 
          , ("RRK2", "Runge_Kutta_Rationnel_ordre_2", "1.5")
          , ("RK2", "Runge_Kutta_ordre_2_classique", "1.5")
          , ("RK3", "Runge_Kutta_ordre_3_classique", "1.5")
          , ("RK4", "Runge_Kutta_ordre_4_classique", "1.5")
          #, ("RK2", "Runge_Kutta_ordre_2", "1.5")
          ]

N_VDF = ["8 22", "16 8", "16 16"]
N_VEF = ["4 11", "8 4", "8 8"]

for d, p, _ in dis:
    for sc, sch, fsc in schema:
        for ex, di in diffusion:
            if d == "VDF":
                run.addCaseFromTemplate("jdd.data", f"{d}/{sc}/{ex}", {"trian" : p, "dis" : d, "diff" : di, "sch" : sch, "fsc" : fsc, "N1": "7 21", "N2" : "15 7", "N3" : "15 15"})
            else:
                run.addCaseFromTemplate("jdd.data", f"{d}/{sc}/{ex}", {"trian" : p, "dis" : d, "diff" : di, "sch" : sch, "fsc" : fsc, "N1": "4 11", "N2" : "8 4", "N3" : "8 8"})
        
run.printCases()
run.runCases()
run.tablePerf()

# Convergence

In [ ]:
from trustutils import plot
import numpy as np

a = plot.Graph(label_size=16, title_size=24, legend_size=12, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Convergence: {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd.dt_ev", label=f"{d}/{sc}", marker=m, linewidth=(2 if d == "VDF" else 1))
    a.scale(yscale="log")

# Time step evolution (in seconds)

In [ ]:
a = plot.Graph(label_size=16, title_size=24, legend_size=12, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Time steps: {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            dt_ev = np.loadtxt(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd.dt_ev")
            a.add(dt_ev[:, 0], dt_ev[:, 1], label=f"{d}/{sc}", marker=m)

# Vertical profiles 

In [ ]:
a = plot.Graph(label_size=16, title_size=24, legend_size=12, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Flux pb1 vs pb2: {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            fpb2 = np.loadtxt(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd_pb2_Diffusion_chaleur.out")
            a.add(fpb2[:, 0], fpb2[:, 4], label=f"{d}/{sc} - pb 2", marker=m, linewidth=(2 if d == "VDF" else 1) )
    
            fpb1 = np.loadtxt(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd_pb1_Diffusion_chaleur.out")
            a.add(fpb1[:, 0], -1*fpb1[:, 3], label=f"{d}/{sc} pb 1", marker=m, linewidth=(2 if d == "VDF" else 1))

